<a href="https://colab.research.google.com/github/thoadao0301/AI-Project/blob/NamTP/Implement_Keras_Facenet_Vietnam_Famous_Celebrities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>